In [1]:
class tensor:
    """
    Refer The below link to know why mutable datasetrutures cant be in the initialization
    https://stackoverflow.com/questions/4841782/python-constructor-and-default-value
    """
    def __init__(self,value,parent=None,child=None,operation=None,grad=None,gradstatus =0):
        self.value = value
        self.grad = grad
        self.operation = operation
        self.gradstatus = 0
        if child is None:
            self.child = []
        else:
            self.child = child
        if parent is None:
            self.parent = []
        else:
            self.parent = parent
            
        
    def get_value(self):
        return self.value
    def get_consumer(self):
        return self.child
    def get_parent(self):
        return self.parent
    def get_operation(self):
        return self.operation
    def get_grad(self):
        return self.grad
    def get_gradstatus(self):
        return self.gradstatus
    
    def update_child(self,child):
        self.child.append(child)
    def update_value(self,value):
        self.value = value
    def update_grad(self,grad):
        self.grad += grad
   
    def set_grad(self,grad):
        self.grad = grad
    def set_gradstatus(self,stat):
        self.gradstatus = stat
        
    def size(self):
        return 1
    def __repr__(self):
        return 'tensor object:'+ str(id(self)) 

class operation:
    def mult(a,b,bprop=False,grad = None):
        a_val = a.get_value()
        b_val = b.get_value()
        
        if bprop == False:
            res  =  tensor(value = a_val*b_val,parent =[a,b],operation=operation.mult)
            a.update_child(res)
            b.update_child(res)
         
        else:
            res = (b_val*grad,a_val*grad)
          
        
        return res

    def add(a,b,bprop=False,grad = None):
        a_val = a.get_value()
        b_val = b.get_value()
        
        if bprop == False:
            res  =  tensor(value = a_val+b_val,parent = [a,b],operation=operation.add)
            a.update_child(res)
            b.update_child(res)
        else:
            res = (1*grad,1*grad)
          
        return res
    
    def sub(a,b,bprop=False,grad = None):
        a_val = a.get_value()
        b_val = b.get_value()
        
        if bprop == False:
            res  =  tensor(value = a_val-b_val,parent =[a,b],operation=operation.sub)
            a.update_child(res)
            b.update_child(res)
        else:
            res = (1*grad,-1*grad)
          
        return res
    
    def div(a,b,bprop=False,grad = None):
        a_val = a.get_value()
        b_val = b.get_value()
        
        if bprop == False:
            res  =  tensor(value = a_val/b_val,parent =[a,b],operation=operation.div)
            a.update_child(res)
            b.update_child(res)
        else:
            res = (1/b_val*grad,-a_val/b_val**2*grad)
          
        return res
  

class graph:
    def __init__(self,func,var=None):
        self.func = func
        if var is None:
            self.var = []
        else:
            self.var = var
        self.func_anc = []
        self.var_desc = []
        self.gr = [] #final graph
        self.tmp = [] # stores cache nodes of desc and anc
    
    def get_ancestors(self,var):
        self.tmp=[]
        self.__get_ancestors__(var)
        return self.tmp
    
    def get_descendents(self,var):
        self.tmp=[]
        self.__get_descendents__(var)
        return self.tmp
    
    def get_graph(self):
        if len(self.gr) is 0:
            self.__get_graph__()
        return self.gr
    
    def get_grad(self):
        self.__get_graph__()   
        self.__grad_init__()
        self.__get_grad__(self.var)
        grad_table =[]
        for v in self.var:
            grad_table += [v.get_grad()]
        return grad_table
        
    
    def __get_grad__(self,var):
        for v in var:
            if v.get_gradstatus() is 0:
                child = self.__get_consumer__(v)
                #if child.size:
                for ch in child:
                    op = ch.get_operation()
                    par =  self.__get_parent__(ch)
                    pa1,pa2 = ch.get_parent()
                    self.__get_grad__([ch])
                    pa1_grad_temp,pa2_grad_temp = op(pa1,pa2,True,ch.get_grad())
                    if pa1 == v:
                        v.update_grad(pa1_grad_temp)
                    elif pa2 == v:
                        v.update_grad(pa2_grad_temp)

            v.set_gradstatus(1)            
                    
    
    def __get_graph__(self):
        if len(self.gr) is 0:
            self.__var_union__()
            self.__func_anc__()
            self.gr = list(set(self.func_anc)&set(self.var_desc)|set(self.var)|set([self.func]))
        
        
    def __grad_init__(self):
        for nodes in self.gr:
            nodes.set_grad(0)
            nodes.set_gradstatus(0)
        self.func.set_grad(1)
        self.func.set_gradstatus(1)

                    
    
    def __get_ancestors__(self,var):
        parent = var.get_parent()
        for p in parent:
            self.tmp.append(p)
            if p.size :
                self.__get_ancestors__(p)
               
    
    def __get_descendents__(self,var):
        child = var.get_consumer()
        for ch in child:
            self.tmp.append(ch)
            if ch.size:
                self.__get_descendents__(ch)
        
    
    def __var_union__(self):
        for v in self.var:
            self.var_desc = list(set(self.var_desc)|set(self.get_descendents(v)))
   
    def __func_anc__(self):
        self.func_anc = self.get_ancestors(self.func)
        
    def __get_consumer__(self,v):
        tmp_res = v.get_consumer()
        return (list(set(tmp_res)&set(self.gr)))
    
    def __get_parent__(self,v):
        tmp_res = v.get_parent()
        return (list(set(tmp_res)&set(self.gr)))
      

In [5]:
# Testing our backpropagation code
op = operation # Math operation class

# Input initialzation
a = tensor(value = 5)
b = tensor(value = 6)
c = tensor(value = 8)
d = tensor(value = 11)
# Performing few operation
e = op.mult(a,b)
f = op.mult(c,d)
g = op.div(e,f)#
h = op.div(tensor(1),g)

gr = graph(func = g,var = [a,b])

# gr.get_graph() #- Gives you the pruned graph

# Getting gradients of g wrt (a and b)
print('Test 1')
print('Derived through Bakpropagation :',gr.get_grad())
print('Actual gradients :',6/8/11,5/8/11)
print('\n')
# Getting gradients of h wrt (a and b)
print('Test 2')
gr = graph(func = h,var = [a,b])
gr.get_graph()
print('Derived through Bakpropagation :',gr.get_grad())
print('Actual gradients :',-(6/8/11)/(30/88)**2,-(5/8/11)/(30/88)**2)



Test 1
Derived through Bakpropagation : [0.06818181818181818, 0.05681818181818182]
Actual gradients : 0.06818181818181818 0.056818181818181816


Test 2
Derived through Bakpropagation : [-0.5866666666666667, -0.48888888888888893]
Actual gradients : -0.5866666666666667 -0.48888888888888893
